___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Part One: Understanding Order Types

Of course there are more possibilities to buy an equity besides simply using **SetHoldings**. <br />
In fact, quantconnect provides the following orders (https://www.quantconnect.com/docs/algorithm-reference/trading-and-orders#Trading-and-Orders-Placing-Orders):

* **MarketOrder(symbol, num_shares)**: Filled as soon as possible at the current price
* **LimitOrder(symbol, num_shares, limit_price)**: Filled when the price drops below (when buying) or above (when selling) the limit price
* **StopMarketOrder(symbol, num_shares, stop_price)**: Filled when the price rises above the stop_price (when buying) or below (when selling)
* **StopLimitOrder(symbol, num_shares, stop_price, limit_price)**: Create a limit order when the stop_price is reached. Then behaves as the usual LimitOrder
* **MarketOnOpenOrder(symbol, num_shares)**: Filled at the opening price (i.e when the market opens). Need to be defined before the market opens
* **MarketOnCloseOrder(symbol, num_shares)**: Filled at the closing price (i.e when the market closes)

## MarketOrder for 1000 Apple shares.
Here we simply want to buy 1000 Apple shares at the current market price at the 1.1.2010 and sell them at the 1.1.2020

In [ ]:
class BuyApple(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 1)  # Start on 1.1.2010
        self.SetCash(10000)  #  10,000$ are available
        
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.invest_toggle = True

    def OnData(self, data):
        
        
        if  not self.Portfolio.Invested and self.invest_toggle:
            # Buy 1000 Apple shares
            self.MarketOrder(self.apple.Symbol, 1000)
            self.invest_toggle = False
            
        if self.Time == datetime(day=1,month=1,year=2020):
            self.MarketOrder(self.apple.Symbol, -1000)
    


## LimitOrder for 50 IBM shares.
Here we create a limit Order to buy 50 IBM shares for a max price of 50\\$ on the 1.1.2008 ( the order is filled later). <br />
Aditionally we create another limit order at least 150\\$ to sell all shares at a price of 150\\$

In [ ]:
class BuyIBMLimit(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2008, 1, 1)  # Start on 1.1.008
        self.SetCash(10000)  #  10,000$ are available
        
        self.ibm = self.AddEquity("IBM", Resolution.Daily)
        self.invest_toggle = True

    def OnData(self, data):
        
        # Create the limit orders
        if  not self.Portfolio.Invested and self.invest_toggle:
            # Create a limit order to buy 50 IBM shares for a max of $50
            self.LimitOrder(self.ibm.Symbol, 50, 50)
            
            # Create a limit order to sell 50 IBM shares for a min of $150
            self.LimitOrder(self.ibm.Symbol, -50, 150)
            self.invest_toggle = False
            
    


## StopMarketOrder for 50 IBM stocks.
A Stop market order waits until a stop price has been reached, and then initiates a Market Order if that stop price is reached. 
To demonstrate this we buy 50 IMB shares on the 1.1.2013 and to reduce possible losses, we create a StopMarketOrder for those shares at 120\\$

In [ ]:
class StopMarketOrderBA(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2019, 1, 1)  
        self.SetCash(100000)  #  100,000$ are available
        
        self.ba = self.AddEquity("BA", Resolution.Daily)
        self.sell_ticket = None
        self.invest_toggle = True

    def OnData(self, data):
        
        # Buy the shares and create the StopMarketOrder
        if  not self.Portfolio.Invested and self.invest_toggle:
            # Buy 50 IBM shares
            self.MarketOrder(self.ba.Symbol, 50)
            self.invest_toggle = False
            
        if self.sell_ticket == None:   
            # Create the stop market order to sell 50 BA Shares if it drops below 60% purchase
            self.sell_ticket = self.StopMarketOrder(self.ba.Symbol, -50, self.Securities['BA'].Open*0.6,'Sell BA')
            

## StopLimitOrder for 50 IBM stocks.
To demonstrate the StopLimitOrder we create a stop limit order on the 1.1.2013 with a stop_price of 120\\$ and a limit price of 110\\$. The Limit Order is created when the price drops below 120\\$ and filled when the price rises above 130\\$

In [ ]:
class SPYStopLimit(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)  # Set Start Date
        self.SetCash(100000)  # Set Strategy Cash
        self.spy =  self.AddEquity("SPY", Resolution.Daily)
        

    def OnData(self, data):
        '''OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.
            Arguments:
                data: Slice object keyed by symbol containing the stock data
        '''

        if not self.Portfolio.Invested:
            
            close = self.Securities["SPY"].Close
            stopPrice = close * .99 # Trigger stop limit when price falls 1%.
            limitPrice = close * 1.01 # Buy equal or better than 1% > close.
            stopLimitTicket = self.StopLimitOrder("SPY", 10, stopPrice, limitPrice)

## MarketOnOpenOrder for 1000 Apple stocks
Let's buy 1000 apple stocks with the market open order. Feel free to try using the OnClose Order

In [ ]:
class BuyAppleOpen(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2010, 1, 1)  # Start on 1.1.2010
        self.SetCash(10000)  #  10,000$ are available
        
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.invest_toggle = True

    def OnData(self, data):
        
        # Buy and hold forever
        if  not self.Portfolio.Invested and self.invest_toggle:
            # Buy 1000 Apple shares on market opening
            self.MarketOnOpenOrder(self.apple.Symbol, 1000)
            self.invest_toggle = False

## Getting Price and Share Information

**If price rises 10% from initial purchase price, sell 50% of holdings. Only do this action once**

1. Figure out how to get the initial purchase price
2. Figure out Daily price
3. Figure out how many shares we hold

In [ ]:
class BuyAndSellApple(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2015, 1, 1)  # Start on 1.1.2015
        self.SetEndDate(2020, 1, 1)  # Start on 1.1.2015
        self.SetCash(10000)  #  10,000$ are available
        
        self.apple = self.AddEquity("AAPL", Resolution.Daily)
        self.invest_toggle = True
        self.sell_toggle = True

    def OnData(self, data):
        
        # Return if data is not available
        if not data[self.apple.Symbol]:
            return

        if not self.Portfolio.Invested and self.invest_toggle:
            
            # Buy as many shares as possible
            # Same as SetHoldings command
            shares_to_buy = int(self.Portfolio.Cash / data[self.apple.Symbol].Open)
            
            self.MarketOrder(self.apple.Symbol, shares_to_buy)
            self.invest_toggle = False
            return
        
        # Compute proft in percent
        profit = self.Portfolio[self.apple.Symbol].UnrealizedProfitPercent
        
        # If profit is greater than 10% we sell 50% of our stocks
        if profit >= 0.1 and self.sell_toggle:
            
            # Get quantity of apple shares in portfolio
            held_stocks = self.Portfolio[self.apple.Symbol].Quantity
            self.MarketOrder(self.apple.Symbol, -(held_stocks // 2))
            
            self.sell_toggle = False

    
    def OnOrderEvent(self, OrderEvent):
        # Event when the order is filled. Debug log the order fill. :OrderEvent:

        if OrderEvent.FillQuantity == 0:
            return

        fetched = self.Transactions.GetOrderById(OrderEvent.OrderId)

        self.Log(f"{str(fetched.Type)} was filled. Symbol: {str(OrderEvent.Symbol)}. Quantity: {str(OrderEvent.FillQuantity)}. Direction: {str(OrderEvent.Direction)}")


# Part Two: Interacting with Order Ticket Objects

Now that we understand the logic behind each order type, let's explore how to interact with the OrderTicket object, including canceling and updating orders.

## Premonition of Citigroup Crisis

In this example, you are back in the Financial Crisis, invested in Citigroup on Jan 1st 2007. You feel uneasy about the future however, so you decide to place a **StopMarketOrder** at 50% of your initial closing price for C. This means that if in the future, your investment drops by 50% in value, you will cut your losses and initiate a Market Order to exit your position at the market price.

We'll also explore these addtional concepts:

* Data Normalization Options
* OnEndOfDay Method
* OnOrderEvent Method

In [ ]:
class PremonitionOfCrisis(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(100000)
        
        # Financial Crisis
        self.SetStartDate(2007,1,1)
        self.SetEndDate(2010,1,1)
        
        
        self.citi = self.AddEquity("C", Resolution.Daily) 
        # https://www.quantconnect.com/docs/v2/writing-algorithms/user-guides/assets-and-datasets/data-normalization
        self.citi.SetDataNormalizationMode(DataNormalizationMode.Raw)
        self.c = self.citi.Symbol
        
        self.sellTicket = None
        self.invest = True
        self.loss_limit = 0.5

    def OnData(self, data):
        
        if not self.Portfolio.Invested and self.invest:
            # Initial Market Order for C
            self.initial_order = self.MarketOrder(self.c, 10)
            self.invest = False
        
        
        if self.sellTicket is None:
            # Initiate a Market Order if Stop of 50% loss is hit.
            self.sellTicket = self.StopMarketOrder(
                self.c,
                -10,
                self.Securities["C"].Close * self.loss_limit,
                "Loss Limit Stop")

    # Try Typing
    # def On to see auto-suggestions of available Method Triggers

    def OnEndOfDay(self):
        '''End of each day, report the current Close Price'''
        if self.sellTicket is None:
            return

        else:
            self.Log(str(self.Securities["C"].Close))

    def OnOrderEvent(self, OrderEvent):
        # Event when the order is filled. Debug log the order fill. :OrderEvent:

        if OrderEvent.FillQuantity == 0:
            return
        
         

        fetched = self.Transactions.GetOrderById(OrderEvent.OrderId)

        self.Log(f"{str(fetched.Type)} was filled. Symbol: {str(OrderEvent.Symbol)}. Quantity: {str(OrderEvent.FillQuantity)}. Direction: {str(OrderEvent.Direction)}")


## Updating An Order

In [ ]:
class PremonitionOfCrisisUpdate(QCAlgorithm):

    def Initialize(self):
        
        self.SetCash(100000)
        
        # Financial Crisis
        self.SetStartDate(2007,1,1)
        self.SetEndDate(2010,1,1)
        
        
        self.citi = self.AddEquity("C", Resolution.Daily) 
        # https://www.quantconnect.com/docs/v2/writing-algorithms/user-guides/assets-and-datasets/data-normalization
        self.citi.SetDataNormalizationMode(DataNormalizationMode.Raw)
        self.c = self.citi.Symbol
        self.first_day_close = None
        self.sellTicket = None
        self.invest = True
        self.loss_limit = 0.5

    def OnData(self, data):
        
        if not self.Portfolio.Invested and self.invest:
            # Initial Market Order for VNQ
            self.initial_order = self.MarketOrder(self.c, 10)
            self.first_day_close = self.Securities["C"].Close
            self.invest = False
        
        
        if self.sellTicket is None:
            # Initiate a Market Order if Stop of 50% loss is hit.
            self.sellTicket = self.StopMarketOrder(
                self.c,
                -10,
                self.Securities["C"].Close * self.loss_limit,
                "Loss Limit Stop")

    # Try Typing
    # def On to see auto-suggestions of available Method Triggers

    def OnEndOfDay(self):
        '''End of each day, report the current Close Price'''
        if self.sellTicket is None:
            return

        else:
            self.Log(str(self.Securities["C"].Close))
        
        # If there is any day where the stock open/close difference was more
        # than 5%, let's update our Stop limit for the Market order
        if ((self.Securities["C"].Open - self.Securities['C'].Close)/self.Securities["C"].Open ) > 0.05:
            
            # STEP 1: Get the Order Fields
            updateSettings = UpdateOrderFields()
            # STEP 2: Grab the porperty to update (check docs for var names)
            # Here we update the StopPrice
            updateSettings.StopPrice = self.first_day_close * 0.75
            # STEP 3: Initiate the Update
            response = self.sellTicket.Update(updateSettings)
            
            if response.IsSuccess:
                self.Debug("Order updated successfully")

    def OnOrderEvent(self, OrderEvent):
        # Event when the order is filled. Debug log the order fill. :OrderEvent:

        if OrderEvent.FillQuantity == 0:
            return
        
         

        fetched = self.Transactions.GetOrderById(OrderEvent.OrderId)

        self.Log(f"{str(fetched.Type)} was filled. Symbol: {str(OrderEvent.Symbol)}. Quantity: {str(OrderEvent.FillQuantity)}. Direction: {str(OrderEvent.Direction)}")
